<a href="https://colab.research.google.com/github/HyeonKj/TIL/blob/master/Testing_TPOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TPOT test

In [ ]:
import time
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive #드라이브에서 파일 가지고 오기 
drive.mount('/content/drive')

#불러올 파일의 경로를 filename 변수에 저장
filename = '/content/drive/MyDrive/전복나이예측대회/dataset/train.csv'

#pandas read_csv로 불러오기
inputData = pd.read_csv(filename)
display(inputData.sample(10))

In [ ]:
# 추후 원핫인코딩으로 변경 

sexEncoded = []
for i in range(len(inputData["Gender"].values)):
    if inputData["Gender"].values[i] == "M":
        value=1
    elif inputData["Gender"].values[i] == "F":
        value=0
    else:
        value=-1
    sexEncoded.append(value)
inputData["Gender"] = sexEncoded

display(inputData.describe())

In [ ]:
inputData.shape

In [ ]:
inputData.dtypes

In [ ]:
print(inputData.info())

In [ ]:
# load test dataset
test_df = pd.read_csv('/content/drive/MyDrive/전복나이예측대회/dataset/test.csv')
test_df.drop(['id'], axis=1, inplace=True)

# transform the gender feature as a onehot encoded value
test_df = pd.get_dummies(test_df)
test_df.head()

# [데이콘 공유된 코드 참고](https://dacon.io/competitions/official/235877/codeshare/4698?page=1&dtype=recent)


In [ ]:
y_train = inputData['Target']
# convert to numpy array
X_train = inputData.values
y_train = y_train.values
X_test = test_df.values
print(X_train.shape, y_train.shape, X_test.shape)

# (1253, 10) (1253,) (2924, 10)

In [ ]:
from sklearn.preprocessing import StandardScaler

# scale features - standard scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# scale target - minmax + median normalization
# -> to be zero centered
y_train_scaled = (y_train - y_train.min()) * 2 / (y_train.max() - y_train.min())
median = np.median(y_train_scaled)
y_train_scaled = y_train_scaled - median

In [ ]:
# histogram of scaled target
plt.hist(y_train_scaled, bins=20)
plt.title('Histogram of scaled target')
plt.text(1, 70, f'Mean: {y_train_scaled.mean():.2f}')
plt.text(1, 50, f'Std: {y_train_scaled.std():.2f}')
plt.text(1, 30, f'Median: {np.median(y_train_scaled):.2f}')
plt.show()

In [ ]:
from tensorflow import keras

weight_decay = 1e-5
learning_rate = 1e-3

# build a simple FNN using keras
model = keras.Sequential([
    keras.layers.Dense(128, activation='tanh', input_shape=(X_train.shape[1],), kernel_regularizer=keras.regularizers.l2(weight_decay)),
    # keras.layers.BatchNormalization(),
    keras.layers.Dense(64, activation='tanh', kernel_regularizer=keras.regularizers.l2(weight_decay)),
    # keras.layers.BatchNormalization(),
    keras.layers.Dense(32, activation='tanh' ,kernel_regularizer=keras.regularizers.l2(weight_decay)),
    # keras.layers.BatchNormalization(),
    keras.layers.Dropout(0),
    keras.layers.Dense(1)
])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate) , loss='mae', metrics=['mae'])
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)
lr_scheduler = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

In [ ]:
history = model.fit(X_train_scaled, y_train_scaled, validation_split=0.1,  epochs=1000, batch_size=128, callbacks=[early_stopping, lr_scheduler], verbose=0)
plt.plot(history.history['loss'][3:], label='train')
plt.plot(history.history['val_loss'][3:], label='val')
plt.legend()
plt.show()
print(history.history['val_loss'][-30])

In [ ]:
# predict test data
y_hat = model.predict(X_test_scaled)

In [ ]:
# reverse scaling
y_hat_unscaled = (y_hat + median) / 2 * (y_train.max() - y_train.min()) + y_train.min()
y_hat_unscaled = y_hat_unscaled.astype(int)
y_hat_unscaled = np.clip(y_hat_unscaled, 1, 100)

In [ ]:
# histogram of scaled target
plt.hist(y_hat_unscaled, bins=10, label='predicted', alpha=0.5)
plt.hist(y_train, bins=10, label='original', alpha=0.5)
plt.title('Histogram of predicted and original target')
plt.legend()
plt.show()

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/전복나이예측대회/dataset/sample_submission.csv')
submission['Target'] = y_hat_unscaled
submission.to_csv('submission.csv', index=False)
#The End

In [ ]:
submission.shape

In [ ]:
submission.info()

In [ ]:
submission.describe()

In [ ]:
y_hat_unscaled.values

In [ ]:
y = inputData["Target"]
X = inputData.copy(deep=True)
X.drop(["Target"], axis=1, inplace=True)

scaler = MaxAbsScaler()
scaler.fit(X)
X = scaler.transform(X)
X = pd.DataFrame(X)

# **코드**  [TPOT 사용](https://www.simonwenkel.com/2019/06/08/revisitingML-abalone.html)

In [ ]:
!pip install tpot

In [ ]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error, mean_squared_error, mean_absolute_error
import sklearn.metrics
from tpot import TPOTRegressor


In [ ]:
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_squared_log_error, median_absolute_error
import sklearn.metrics
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
# from skgarden.mondrian import MondrianForestRegressor
import xgboost


In [ ]:

inputData = pd.read_csv('/content/drive/MyDrive/전복나이예측대회/dataset/train.csv')
sexEncoded = []
for i in range(len(inputData["Gender"].values)):
    if inputData["Gender"].values[i] == "M":
        value=1
    elif inputData["Gender"].values[i] == "F":
        value=0
    else:
        value=-1
    sexEncoded.append(value)
inputData["Gender"] = sexEncoded

y = inputData["Target"]
X = inputData.copy(deep=True)
X.drop(["Target"], axis=1, inplace=True)

scaler = MaxAbsScaler()
scaler.fit(X)
X = scaler.transform(X)
X = pd.DataFrame(X)

X_train = inputData.values
X_test = test_df.values
y_train = inputData['Target']
y_test = y_hat_unscaled

tpot = TPOTRegressor(max_time_mins=180,
                     verbosity=1,
                     n_jobs=-1)
tpot.fit(X_train,y_train)
tpot.export('abalone.py')

y_predictions = tpot.predict(X_test)
r2 = sklearn.metrics.r2_score(y_test, y_predictions)
mae = sklearn.metrics.mean_absolute_error(y_test, y_predictions)
mse = sklearn.metrics.mean_squared_error(y_test, y_predictions)
rmse = np.sqrt(mse)
print("R2 score:", r2)
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)